In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import LSTM
from sklearn.base import BaseEstimator, RegressorMixin
from keras.models import Sequential
from tensorflow.keras.models import Model
from keras.layers import Input, GRU, LSTM, Dense, Dropout, Concatenate, Reshape
from keras.optimizers import Adam
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
train = pd.read_csv('train_Minterpolate.csv')
train.head()

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
0,A,5,1,9,STN001,3,A,5,1,12,...,0,0,0,0,0,0,0,0,0.0,0
1,A,5,1,9,STN001,6,A,5,1,15,...,0,0,0,0,0,0,0,0,0.0,0
2,A,5,1,9,STN001,9,A,5,1,18,...,0,0,0,0,0,0,0,0,0.0,0
3,A,5,1,9,STN001,12,A,5,1,21,...,0,0,0,0,0,0,0,0,0.0,0
4,A,5,1,9,STN001,15,A,5,2,0,...,0,0,0,0,0,0,0,0,0.0,0


In [18]:
test = pd.read_csv('rainfall_test.csv')
test.head()

,Unnamed: 0,rainfall_test.fc_year,rainfall_test.fc_month,rainfall_test.fc_day,rainfall_test.fc_hour,rainfall_test.stn4contest,rainfall_test.dh,rainfall_test.ef_year,rainfall_test.ef_month,rainfall_test.ef_day,...,rainfall_test.v01,rainfall_test.v02,rainfall_test.v03,rainfall_test.v04,rainfall_test.v05,rainfall_test.v06,rainfall_test.v07,rainfall_test.v08,rainfall_test.v09,rainfall_test.class_interval
0,1,D,5,1,9,STN031,3,D,5,1,...,68,46,13,1,0,0,0,0,0,NaN
1,2,D,5,1,9,STN031,6,D,5,1,...,83,66,26,5,0,0,0,0,0,NaN
2,3,D,5,1,9,STN031,9,D,5,1,...,17,6,0,0,0,0,0,0,0,NaN
3,4,D,5,1,9,STN031,12,D,5,1,...,0,0,0,0,0,0,0,0,0,NaN
4,5,D,5,1,9,STN031,15,D,5,2,...,0,0,0,0,0,0,0,0,0,NaN


In [3]:
df2 = test.iloc[:, :19]

df2['ef_year_temp'] = df2['ef_year'].replace({'A': '2020', 'B': '2021', 'C': '2022'})
df2['ef_datetime'] = pd.to_datetime(df2[['ef_year_temp', 'ef_month', 'ef_day', 'ef_hour']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d-%H')
df2['weight'] = 1 / df2['dh']

for var in ['v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09']:
    df2[f'{var}_weighted'] = df2[var] * df2['weight']

weighted_df2 = df2.groupby(['stn4contest', 'ef_datetime']).apply(
    lambda x: pd.Series({
        'v01': np.average(x['v01'], weights=x['weight']),
        'v02': np.average(x['v02'], weights=x['weight']),
        'v03': np.average(x['v03'], weights=x['weight']),
        'v04': np.average(x['v04'], weights=x['weight']),
        'v05': np.average(x['v05'], weights=x['weight']),
        'v06': np.average(x['v06'], weights=x['weight']),
        'v07': np.average(x['v07'], weights=x['weight']),
        'v08': np.average(x['v08'], weights=x['weight']),
        'v09': np.average(x['v09'], weights=x['weight'])
    })
).reset_index()

train_y = train.groupby(['stn4contest', 'ef_year', 'ef_month', 'ef_day', 'ef_hour'])[['vv', 'class_interval']].mean().reset_index(drop=True)

weighted_df['vv'] = train_y['vv']
weighted_df['class_interval'] = train_y['class_interval']
weighted_df = weighted_df.astype({'class_interval': 'int'})

weighted_df['year'] = weighted_df['ef_datetime'].dt.year
weighted_df['month'] = weighted_df['ef_datetime'].dt.month
weighted_df['day'] = weighted_df['ef_datetime'].dt.day
weighted_df['hour'] = weighted_df['ef_datetime'].dt.hour

encoder = LabelEncoder()
stn_label = encoder.fit_transform(weighted_df['stn4contest'])
weighted_df['stn4contest'] = stn_label
weighted_df = weighted_df.drop(columns=['ef_datetime'])

data = weighted_df
data.head()

,stn4contest,v01,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval,year,month,day,hour
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,12
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,15
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,18
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,21
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,2,0


In [3]:
df = train.iloc[:, :19]

df['ef_year_temp'] = df['ef_year'].replace({'A': '2020', 'B': '2021', 'C': '2022'})
df['ef_datetime'] = pd.to_datetime(df[['ef_year_temp', 'ef_month', 'ef_day', 'ef_hour']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d-%H')
df['weight'] = 1 / df['dh']

for var in ['v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09']:
    df[f'{var}_weighted'] = df[var] * df['weight']

weighted_df = df.groupby(['stn4contest', 'ef_datetime']).apply(
    lambda x: pd.Series({
        'v01': np.average(x['v01'], weights=x['weight']),
        'v02': np.average(x['v02'], weights=x['weight']),
        'v03': np.average(x['v03'], weights=x['weight']),
        'v04': np.average(x['v04'], weights=x['weight']),
        'v05': np.average(x['v05'], weights=x['weight']),
        'v06': np.average(x['v06'], weights=x['weight']),
        'v07': np.average(x['v07'], weights=x['weight']),
        'v08': np.average(x['v08'], weights=x['weight']),
        'v09': np.average(x['v09'], weights=x['weight'])
    })
).reset_index()

train_y = train.groupby(['stn4contest', 'ef_year', 'ef_month', 'ef_day', 'ef_hour'])[['vv', 'class_interval']].mean().reset_index(drop=True)

weighted_df['vv'] = train_y['vv']
weighted_df['class_interval'] = train_y['class_interval']
weighted_df = weighted_df.astype({'class_interval': 'int'})

weighted_df['year'] = weighted_df['ef_datetime'].dt.year
weighted_df['month'] = weighted_df['ef_datetime'].dt.month
weighted_df['day'] = weighted_df['ef_datetime'].dt.day
weighted_df['hour'] = weighted_df['ef_datetime'].dt.hour

encoder = LabelEncoder()
stn_label = encoder.fit_transform(weighted_df['stn4contest'])
weighted_df['stn4contest'] = stn_label
weighted_df = weighted_df.drop(columns=['ef_datetime'])

data = weighted_df
data.head()

,stn4contest,v01,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval,year,month,day,hour
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,12
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,15
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,18
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,1,21
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2020,5,2,0


In [5]:
features = data[['stn4contest', 'year', 'month', 'day', 'hour', 'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09']]
target = data['vv']

train_data = data[data['stn4contest'] <= 15]
test_data = data[data['stn4contest'] > 15]

X_train = train_data[['stn4contest', 'year', 'month', 'day', 'hour', 'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09']]
y_train = train_data['vv']

X_test = test_data[['stn4contest', 'year', 'month', 'day', 'hour', 'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09']]
y_test = test_data['vv']

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

#y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
#y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [13]:
# 입력 데이터의 형태 정의 (예: 특성 개수는 X_train.shape[1])
input_shape = (X_train_scaled.shape[1],)

# 입력층 정의
inputs = Input(shape=input_shape)

# Reshape 레이어로 입력 데이터를 3D 텐서로 변환
reshaped_inputs = Reshape((input_shape[0], 1))(inputs)

# GRU 레이어 정의
gru_output = GRU(64, return_sequences=False)(reshaped_inputs)

# LSTM 레이어 정의
lstm_output = LSTM(64, return_sequences=False)(reshaped_inputs)

# GRU와 LSTM 출력을 결합
combined = Concatenate()([gru_output, lstm_output])

# 최종 출력층 정의 (회귀를 위한 출력층)
outputs = Dense(1)(combined)

# 모델 정의
ensemble_model = Model(inputs, outputs)

# 모델 컴파일
ensemble_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 모델 요약 출력
ensemble_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 14)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape_1 (Reshape)           │ (None, 14, 1)             │               0 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gru_2 (GRU)                   │ (None, 64)                │          12,864 │ reshape_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 64)                │          16,896 │ reshape_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 128)               │               0 │ gru_2[0][0], lstm[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 1)                 │             129 │ concatenate[0][0]          │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 29,889 (116.75 KB)

 Trainable params: 29,889 (116.75 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
ensemble_model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 3.2638e-04 - mae: 0.0068 - val_loss: 3.8324e-04 - val_mae: 0.0059
Epoch 2/50
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 3.0192e-04 - mae: 0.0057 - val_loss: 4.2747e-04 - val_mae: 0.0106
Epoch 3/50
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2.5725e-04 - mae: 0.0052 - val_loss: 3.9285e-04 - val_mae: 0.0046
Epoch 4/50
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2.9769e-04 - mae: 0.0052 - val_loss: 3.9255e-04 - val_mae: 0.0048
Epoch 5/50
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2.6404e-04 - mae: 0.0051 - val_loss: 3.9460e-04 - val_mae: 0.0073
Epoch 6/50
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2.9975e-04 - mae: 0.0052 - val_loss: 3.8701e-04 - val_mae: 0.0051
Epoch 7/50
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2.6376e-04 - mae: 0.0049 - val_loss: 3.8995e-04 - val_mae: 0.0049
Epoch 8/50
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2.5412e-04 - mae: 0.0049 - val_loss: 3.956

In [15]:
# 모델 평가
loss, mae = ensemble_model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 20.9990 - mae: 0.9497
Test Loss: 18.30120849609375, Test MAE: 0.8905093669891357


In [16]:
# 모델 평가
y_pred_scaled = ensemble_model.predict(X_test_scaled)

# 예측 값과 실제 값을 원래 스케일로 복원
y_pred = scaler_y.inverse_transform(y_pred_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance Metrics:")
print(f"{'Metric':<10}{'Value':<20}")
print(f"{'-'*30}")
print(f"{'RMSE':<10}{rmse:<20.4f}")
print(f"{'MAE':<10}{mae:<20.4f}")
print(f"{'MSE':<10}{mse:<20.4f}")
print(f"{'R²':<10}{r2:<20.4f}")

488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Model Performance Metrics:
Metric    Value               
------------------------------
RMSE      3.3746              
MAE       0.8066              
MSE       11.3881             
R²        0.3681              


In [19]:
ensemble_model.save('ensemble_model(gru+lstm).h5')

In [20]:
#rainfall_test. 제거 데이터
submit=pd.read_csv("rainfall_test_618.csv")

#제출용 데이터
realsubmit=pd.read_csv("rainfall_test_618.csv")

In [21]:
submit.columns = ['fc_year','fc_month','fc_day','fc_hour','stn4contest','dh','ef_year','ef_month','ef_day','ef_hour','v01','v02','v03','v04','v05','v06','v07','v08','v09','class_interval']
submit['ef_year'] = submit['ef_year'].replace({'D': '2023'})
submit['ef_datetime'] = pd.to_datetime(submit[['ef_year', 'ef_month', 'ef_day', 'ef_hour']].astype(str).agg('-'.join, axis=1), 
                                       format='%Y-%m-%d-%H')
stn_mapping = {'STN031': 31, 'STN032': 32, 'STN033': 33, 'STN034': 34, 'STN035': 35}
submit['stn4contest'] = submit['stn4contest'].replace(stn_mapping)

submit.head()

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,class_interval,ef_datetime
0,D,5,1,9,31,3,2023,5,1,12,...,46,13,1,0,0,0,0,0,NaN,2023-05-01 12:00:00
1,D,5,1,9,31,6,2023,5,1,15,...,66,26,5,0,0,0,0,0,NaN,2023-05-01 15:00:00
2,D,5,1,9,31,9,2023,5,1,18,...,6,0,0,0,0,0,0,0,NaN,2023-05-01 18:00:00
3,D,5,1,9,31,12,2023,5,1,21,...,0,0,0,0,0,0,0,0,NaN,2023-05-01 21:00:00
4,D,5,1,9,31,15,2023,5,2,0,...,0,0,0,0,0,0,0,0,NaN,2023-05-02 00:00:00


In [22]:
features = ['stn4contest', 'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08',
       'v09', 'ef_year', 'ef_month', 'ef_day', 'ef_hour']
data = submit[features]
data.head()

,stn4contest,v01,v02,v03,v04,v05,v06,v07,v08,v09,ef_year,ef_month,ef_day,ef_hour
0,31,68,46,13,1,0,0,0,0,0,2023,5,1,12
1,31,83,66,26,5,0,0,0,0,0,2023,5,1,15
2,31,17,6,0,0,0,0,0,0,0,2023,5,1,18
3,31,0,0,0,0,0,0,0,0,0,2023,5,1,21
4,31,0,0,0,0,0,0,0,0,0,2023,5,2,0


In [23]:
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(data)

model = load_model('ensemble_model(gru+lstm).h5')

validation_features_scaled_reshaped = np.reshape(features_scaled, (features_scaled.shape[0], features_scaled.shape[1], 1))
predicted_rainfall = model.predict(validation_features_scaled_reshaped)

3813/3813 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step


In [24]:
predicted_rainfall_df = pd.DataFrame(predicted_rainfall, columns=['PredictedRainfall'])
new_data = submit.copy()
new_data['PredictedRainfall'] = predicted_rainfall_df

In [25]:
new_data.head()

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v03,v04,v05,v06,v07,v08,v09,class_interval,ef_datetime,PredictedRainfall
0,D,5,1,9,31,3,2023,5,1,12,...,13,1,0,0,0,0,0,NaN,2023-05-01 12:00:00,0.138795
1,D,5,1,9,31,6,2023,5,1,15,...,26,5,0,0,0,0,0,NaN,2023-05-01 15:00:00,0.172237
2,D,5,1,9,31,9,2023,5,1,18,...,0,0,0,0,0,0,0,NaN,2023-05-01 18:00:00,0.204190
3,D,5,1,9,31,12,2023,5,1,21,...,0,0,0,0,0,0,0,NaN,2023-05-01 21:00:00,0.235361
4,D,5,1,9,31,15,2023,5,2,0,...,0,0,0,0,0,0,0,NaN,2023-05-02 00:00:00,-0.002847


In [26]:
def get_class_interval(rainfall):
    if rainfall < 0.1:
        return 0
    elif 0.1 <= rainfall < 0.2:
        return 1
    elif 0.2 <= rainfall < 0.5:
        return 2
    elif 0.5 <= rainfall < 1.0:
        return 3
    elif 1.0 <= rainfall < 2.0:
        return 4
    elif 2.0 <= rainfall < 5.0:
        return 5
    elif 5.0 <= rainfall < 10.0:
        return 6
    elif 10.0 <= rainfall < 20.0:
        return 7
    elif 20.0 <= rainfall < 30.0:
        return 8
    else:
        return 9

In [27]:
# PredictedRainfall 값을 기준으로 class_interval 열 업데이트
new_data['class_interval'] = new_data['PredictedRainfall'].apply(get_class_interval)
new_data.head()

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v03,v04,v05,v06,v07,v08,v09,class_interval,ef_datetime,PredictedRainfall
0,D,5,1,9,31,3,2023,5,1,12,...,13,1,0,0,0,0,0,1,2023-05-01 12:00:00,0.138795
1,D,5,1,9,31,6,2023,5,1,15,...,26,5,0,0,0,0,0,1,2023-05-01 15:00:00,0.172237
2,D,5,1,9,31,9,2023,5,1,18,...,0,0,0,0,0,0,0,2,2023-05-01 18:00:00,0.204190
3,D,5,1,9,31,12,2023,5,1,21,...,0,0,0,0,0,0,0,2,2023-05-01 21:00:00,0.235361
4,D,5,1,9,31,15,2023,5,2,0,...,0,0,0,0,0,0,0,0,2023-05-02 00:00:00,-0.002847


In [28]:
new_data.class_interval.unique()

array([1, 2, 0], dtype=int64)

In [29]:
prediction_test=new_data.copy() #new_data : 개개인마다 달라유
    
merged = submit.merge(prediction_test[['ef_datetime', 'stn4contest', 'class_interval']], 
                      on=['ef_datetime', 'stn4contest'], 
                      how='left', 
                      suffixes=('', '_pred'))
merged['class_interval_pred']


0          1
1          1
2          2
3          2
4          0
          ..
2379195    1
2379196    1
2379197    1
2379198    2
2379199    2
Name: class_interval_pred, Length: 2379200, dtype: int64

In [30]:
#-999 index 저장
idx999 = realsubmit[realsubmit['rainfall_test.class_interval'].notnull()].index
merged.loc[idx999, 'class_interval_pred'] = -999

#제출용 데이터에 class_interval 채우기
realsubmit['rainfall_test.class_interval']=merged['class_interval_pred']

#저장
realsubmit.to_csv("240494.csv")  # 각자 접수번호.csv 로 저장

In [42]:
realsubmit.head()

,rainfall_test.fc_year,rainfall_test.fc_month,rainfall_test.fc_day,rainfall_test.fc_hour,rainfall_test.stn4contest,rainfall_test.dh,rainfall_test.ef_year,rainfall_test.ef_month,rainfall_test.ef_day,rainfall_test.ef_hour,rainfall_test.v01,rainfall_test.v02,rainfall_test.v03,rainfall_test.v04,rainfall_test.v05,rainfall_test.v06,rainfall_test.v07,rainfall_test.v08,rainfall_test.v09,rainfall_test.class_interval
0,D,5,1,9,STN031,3,D,5,1,12,68,46,13,1,0,0,0,0,0,0
1,D,5,1,9,STN031,6,D,5,1,15,83,66,26,5,0,0,0,0,0,0
2,D,5,1,9,STN031,9,D,5,1,18,17,6,0,0,0,0,0,0,0,0
3,D,5,1,9,STN031,12,D,5,1,21,0,0,0,0,0,0,0,0,0,0
4,D,5,1,9,STN031,15,D,5,2,0,0,0,0,0,0,0,0,0,0,0


##  CSI: 0.01
0.019.